## Create an ai chatbot booking system

# Import required modules
from agents import Agent, Runner
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
import datetime
import os

# Google Calendar setup function
FUNCTION build_calendar_service():
    SCOPES = ['https://www.googleapis.com/auth/calendar.events']
    creds = get_credentials(SCOPES)  # Handles token.json / OAuth flow
    RETURN build('calendar', 'v3', credentials=creds)

# Tool function to create event
FUNCTION create_event(service, event_data):
    # event_data is dict: {'summary': str, 'start': str ISO, 'end': str ISO}
    event = {
        'summary': event_data['summary'],
        'start': {'dateTime': event_data['start'], 'timeZone': 'Australia/Melbourne'},
        'end': {'dateTime': event_data['end'], 'timeZone': 'Australia/Melbourne'}
    }
    service.events().insert(calendarId='primary', body=event).execute()
    RETURN "Event created successfully"

# Define LLM Agent
AGENT booking_agent = Agent(
    name="Event Booker",
    instructions="Parse user message for event: summary, start datetime (ISO), end datetime (ISO). Output JSON only with keys: summary, start, end. Ask for missing details."
)

# Add tool to agent
booking_agent.add_tool(create_event)

# Main booking function
FUNCTION book_event(user_message):
    service = build_calendar_service()
    result = Runner.run_sync(booking_agent, user_message)
    IF result.final_output contains "Event created":
        RETURN "Booking confirmed"
    ELSE:
        RETURN result.final_output  # e.g., clarification needed

# Usage
user_input = "Book team meeting tomorrow 2pm to 3pm"
book_event(user_input)


In [4]:
# Libraries
from openai import OpenAI
from agents import Agent, Runner, function_tool
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from dotenv import load_dotenv
import datetime
import os
import json
from pydantic import BaseModel
import pickle


In [5]:
load_dotenv(override = True)

True

In [6]:
import os
var = os.environ.get('OPENAI_API_KEY')
if var:
    print(var[0:3])

sk-


In [7]:
# Google Calendar Authentication and Setup
def get_calendar_service():
    """
    Authenticates and returns Google Calendar API service.
    
    First time: Opens browser for OAuth, saves token.json
    Later: Uses saved token.json
    """
    SCOPES = ['https://www.googleapis.com/auth/calendar.events']
    creds = None
    
    # Check if token.json exists (saved credentials)
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # If no valid credentials, authenticate
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            # Refresh expired token
            creds.refresh(Request())
        else:
            # First time: run OAuth flow
            # Let the library pick the redirect URI automatically
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', 
                SCOPES
            )
            # Opens browser, you login, returns credentials
            creds = flow.run_local_server(port=8080)
        
        # Save credentials for next time
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    
    return build('calendar', 'v3', credentials=creds)


def create_calendar_event(service, summary, start_time, end_time, description="", location=""):
    """
    Creates an event in Google Calendar.
    
    Args:
        service: Google Calendar service object
        summary: Event title
        start_time: Start datetime (ISO format string or datetime object)
        end_time: End datetime (ISO format string or datetime object)
        description: Event description (optional)
        location: Event location (optional)
    
    Returns:
        Created event details with link
    """
    # Convert datetime objects to ISO format if needed
    if isinstance(start_time, datetime.datetime):
        start_time = start_time.isoformat()
    if isinstance(end_time, datetime.datetime):
        end_time = end_time.isoformat()
    
    event = {
        'summary': summary,
        'location': location,
        'description': description,
        'start': {
            'dateTime': start_time,
            'timeZone': 'Australia/Melbourne',
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'Australia/Melbourne',
        },
    }
    
    # Insert event into calendar
    event_result = service.events().insert(calendarId='primary', body=event).execute()
    
    print(f"✓ Event created: {event_result.get('summary')}")
    print(f"  Link: {event_result.get('htmlLink')}")
    
    return event_result

@function_tool
def book_appointment(event_title, start_datetime, duration_minutes=60,
                     description="", location=""):
    """
    Books an appointment in Google Calendar.
    
    Args:
        event_title: Title of the event
        start_datetime: When the event starts (datetime object)
        duration_minutes: Duration in minutes (default 60)
        description: adding a description to the event
    
    Returns:
        Created event details
    """
    # Get authenticated service
    service = get_calendar_service()
    
    # Calculate end time
    end_datetime = start_datetime + datetime.timedelta(minutes=duration_minutes)
    
    # Create the event
    event = create_calendar_event(
        service=service,
        summary=event_title,
        start_time=start_datetime,
        end_time=end_datetime,
        description=description,
        location=location
    )
    
    return event

In [8]:
# Test the booking system
# Example: Book a meeting tomorrow at 2 PM for 1 hour

tomorrow = datetime.datetime.now() + datetime.timedelta(days=1)
tomorrow_2pm = tomorrow.replace(hour=14, minute=0, second=0, microsecond=0)

try:
    event = book_appointment("Team Meeting", tomorrow_2pm, duration_minutes=60,description="This is really cool event at my place", location="Andersons Creek Road Doncaster East")
    print(f"\n✓ Successfully booked: {event['summary']}")
    print(f"  Start: {event['start']['dateTime']}")
    print(f"  End: {event['end']['dateTime']}")
    print(f"  Description: {event['description']}")
    print(f"  Location: {event['location']}")
except Exception as e:
    print(f"❌ Error: {e}")

❌ Error: 'FunctionTool' object is not callable


In [9]:
# Create AI Booking Agent with OpenAI Agents

# Initialize OpenAI client
client = OpenAI()

# Define the booking tool function
@function_tool
def book_calendar_event(event_title: str, start_datetime: str, description: str = "", location: str = "", duration_minutes: int = 60):
    """
    Books an event in Google Calendar.
    
    Args:
        event_title: The title/name of the event
        start_datetime: Start date and time in ISO format (e.g. "2025-12-04T14:00:00")
        description: Optional description of the event
        location: Optional location of the event
        duration_minutes: Duration in minutes (default 60)
    
    Returns:
        Confirmation message with event details
    """
    # Parse ISO datetime string
    start_dt = datetime.datetime.fromisoformat(start_datetime)
    
    # Get calendar service
    service = get_calendar_service()
    
    # Calculate end time
    end_dt = start_dt + datetime.timedelta(minutes=duration_minutes)
    
    # Create the event
    event = create_calendar_event(
        service=service,
        summary=event_title,
        start_time=start_dt,
        end_time=end_dt,
        description=description,
        location=location
    )
    
    return f"✓ Event '{event['summary']}' booked successfully!\nLink: {event['htmlLink']}"


# Define the booking agent with the tool
booking_agent = Agent(
    name="Calendar Booking Assistant",
    instructions="""You are a helpful calendar booking assistant. 
    
    When a user asks to book an appointment:
    1. Extract the event title/summary
    2. Determine the date and time (handle natural language like "tomorrow at 2pm", "next Monday at 3pm")
    3. Use the book_calendar_event tool to create the event
    4. Return the confirmation message
    
    Be friendly and helpful!""",
    model="gpt-4o",
    tools=[book_calendar_event]
)

print("✓ Booking agent created and ready!")

✓ Booking agent created and ready!


In [13]:
import gradio as gr

# Simple chat function
def chat(message, history):
    try:
        result = Runner.run_sync(booking_agent, message)
        return result.final_output
    except Exception as e:
        return f"Error: {str(e)}"

# Create simple interface with initial greeting
demo = gr.ChatInterface(
    fn=chat,
    title="📅 Oakberry Events Chatbot",
    description="Book Oakberry events through this chatbot",
    chatbot=gr.Chatbot(value=[
        {"role": "assistant", 
         "content": 
         "Hello! I'm your calendar booking assistant for Oakberry Events. I can help you schedule appointments.\n \
         Please tell me: \n \
         - title of event \n \
         - date \n \
         - start and end time of the event \n \
         - location and other notes"}
    ])
)

# Launch
demo.launch(share=True,inline=False)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://a2b36670b2fc045b6f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


✓ Event created: Weeding for 500 people
  Link: https://www.google.com/calendar/event?eid=ZHVxdTU2a2FpM2FjdGZlZzIwcjkyaWMwMzggbWljaGFlbGRpZ2lhbnRvbWFzc29AbQ
